In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import gc
import myutil

from sklearn.ensemble import RandomForestClassifier



In [3]:
def train_model(data_, test_, y_, folds_):

    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    
    feature_importance_df = pd.DataFrame()
    
    feats = [f for f in data_.columns if f not in ['SK_ID_CURR']]
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        
        # Make the random forest classifier
        random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)      
        
        # Train on the training data
        random_forest.fit(trn_x, trn_y)

        # Make predictions
        # Make sure to select the second column only     
        oof_preds[val_idx] = random_forest.predict_proba(val_x)[:, 1]
        sub_preds += random_forest.predict_proba(test_[feats])[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        
        #print(fold_importance_df.shape, log_reg.coef_.shape)
        
        fold_importance_df["importance"] = np.swapaxes(random_forest.feature_importances_,0,1)
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        #print(val_y.shape, oof_preds[val_idx].shape)
        #print(val_y, oof_preds[val_idx])
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
        del log_reg, trn_x, trn_y, val_x, val_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y, oof_preds)) 
    
    test_['TARGET'] = sub_preds

    return oof_preds, test_[['SK_ID_CURR', 'TARGET']], feature_importance_df



KeyboardInterrupt: 

In [ ]:
# Build model inputs
data, test, y = myutil.get_train_test_label()
# Create Folds
folds = KFold(n_splits=5, shuffle=True, random_state=546789)

In [ ]:
oof_preds, test_preds, importances = train_model(data, test, y, folds)

In [ ]:
title = 'randomforest'
myutil.report(test_preds, folds, importances, data, y, oof_preds, title)